In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

/bin/bash: curl: command not found
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 2129 kB of archives.
After this operation, 7662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2129 kB]
Fetched 2129 kB in 2s (947 kB/s)                    
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package git-lfs.
(Reading database ... 33396 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/elastic-7.x.list:1 and /etc/apt/sources.list.d/elastic-7.x.list:2
W: Target Package

In [2]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 52 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.
Filtering content: 100% (2/2), 959.93 MiB | 119.44 MiB/s, done.


In [4]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

In [6]:
tokenizer = SentencePieceBPETokenizer("./kogpt2/vocab.json", "./kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = './kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [7]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [8]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 57 (delta 21), reused 6 (delta 3), pack-reused 18
Unpacking objects: 100% (57/57), done.


In [11]:
import pandas as pd
data = pd.read_csv('./Chatbot_data/ChatbotData.csv')
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [12]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
print(added_special_token_num)

0


In [13]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

3


In [14]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [16]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='./Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [17]:
from torch.utils.data import DataLoader
data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [18]:
from transformers import AdamW

2022-05-19 04:14:11.997368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 04:14:11.997407: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [19]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

epoch no.0  train (0/2956)  loss = 2.99108  avg_loss = 2.99108
epoch no.0  train (200/2956)  loss = 1.25670  avg_loss = 1.47491
epoch no.0  train (400/2956)  loss = 1.21854  avg_loss = 1.29296
epoch no.0  train (600/2956)  loss = 0.87010  avg_loss = 1.22314
epoch no.0  train (800/2956)  loss = 1.17217  avg_loss = 1.19936
epoch no.0  train (1000/2956)  loss = 1.12726  avg_loss = 1.20836
epoch no.0  train (1200/2956)  loss = 1.51246  avg_loss = 1.17679
epoch no.0  train (1400/2956)  loss = 1.01038  avg_loss = 1.16660
epoch no.0  train (1600/2956)  loss = 1.14180  avg_loss = 1.14344
epoch no.0  train (1800/2956)  loss = 0.93434  avg_loss = 1.17536
epoch no.0  train (2000/2956)  loss = 1.00604  avg_loss = 1.16753
epoch no.0  train (2200/2956)  loss = 0.94353  avg_loss = 1.11635
epoch no.0  train (2400/2956)  loss = 1.37008  avg_loss = 1.12345
epoch no.0  train (2600/2956)  loss = 1.15827  avg_loss = 1.12674
epoch no.0  train (2800/2956)  loss = 0.87587  avg_loss = 1.11625
epoch no.1  train

In [20]:
torch.save(model.state_dict(), 'chitchat_model.bin')

In [ ]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [ ]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [ ]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [1]:
import streamlit as st
import torch
from tokenizers import SentencePieceBPETokenizer

def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids, tokenizer):
    return tokenizer.decode_batch(ids)

def get_answer(input_sent, e_s, unk):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [2]:
tokenizer = SentencePieceBPETokenizer("./kogpt2/vocab.json", "./kogpt2/merges.txt")
tokenizer.no_padding()
tokenizer.no_truncation()
e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [3]:
torch.manual_seed(42)
model = torch.load('/opt/ml/Kogpt2_chatbot/chitchat_model.bin', map_location='cuda')

In [ ]:
def get_answer(input_sent, e_s, unk):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)